In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np

In [ ]:
## Dummy comment

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv("/content/7_window_train.csv")

In [ ]:
data.describe()

,Label
count,209672.000000
mean,0.072113
std,0.258675
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [ ]:
import pandas as pd

# Assuming 'data' is your DataFrame
data1 = data.sample(frac=0.5, random_state=42)  # Randomly sample 50% of the data


In [ ]:
data1

,Seq,Label
108089,EKLKTLV,0
199257,MGWKDRW,0
58591,VQEKVNE,0
72667,EILKKRA,0
25625,TNSKSRS,0
...,...,...
82483,GEEKLIA,0
47100,KQAKRHI,0
165948,LVWKVLG,0
71757,KRLKERS,0


In [ ]:
X =  data1['Seq']
y = data1['Label']

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
from collections import Counter

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
from collections import Counter

# Fixed list of 20 standard amino acids
amino_acids = 'ACDEFGHIKLMNPQRSTVWY'

def aac_compute(seq):
    seq = seq.replace('^', '')  # Remove '^' character if present
    aa_counts = Counter(seq)
    total_aa = len(seq)
    aa_percentages = [(aa_counts[aa] / total_aa) * 100 if total_aa > 0 else 0 for aa in amino_acids]
    return [round(v, 2) for v in aa_percentages]  # Round to two decimal places

def evaluate_model(model, X_test, y_test):
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    return accuracy

# Assuming X is a pandas Series containing protein sequences and y is a pandas Series containing labels
# Splitting data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Compute AAC features for each sequence in X_train and X_test
# Convert AAC features to numpy arrays for compatibility with machine learning models
X_train_aac = np.array(X_train.apply(aac_compute).tolist())
X_test_aac = np.array(X_test.apply(aac_compute).tolist())

print("X_train_aac shape:", X_train_aac.shape)
print("y_train shape:", y_train.shape)
print("X_test_aac shape:", X_test_aac.shape)
print("y_test shape:", y_test.shape)

# Define fixed hyperparameters for SVM
svm_params = {'kernel': 'rbf', 'C': 1, 'gamma': 0.01}

# Train SVM model with fixed parameters using GPU
svm_model = SVC(kernel=svm_params['kernel'], C=svm_params['C'], gamma=svm_params['gamma'])
svm_model.fit(X_train_aac, y_train)

# Evaluate the SVM model
svm_accuracy = evaluate_model(svm_model, X_test_aac, y_test)

# Print complete report for SVM
print("\nSVM Model Report:")
print(f"Parameters: {svm_params}")
print(f"Accuracy: {svm_accuracy}")


X_train_aac shape: (83868, 20)
y_train shape: (83868,)
X_test_aac shape: (20968, 20)
y_test shape: (20968,)

SVM Model Report:
Parameters: {'kernel': 'rbf', 'C': 1, 'gamma': 0.01}
Accuracy: 0.9104349484929416


#RF With DPC

In [ ]:
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
from collections import Counter

# Fixed list of 20 standard amino acids
amino_acids = 'ACDEFGHIKLMNPQRSTVWY'

# Function to compute DPC features
def dpc_compute(seq):
    seq = seq.replace('^', '')  # Remove '^' character if present
    dipeptides = [a1 + a2 for a1 in amino_acids for a2 in amino_acids]
    dipeptide_counts = Counter(seq[i:i+2] for i in range(len(seq) - 1))
    total_dipeptides = len(seq) - 1
    dipeptide_percentages = [(dipeptide_counts[dp] / total_dipeptides) * 100 if total_dipeptides > 0 else 0 for dp in dipeptides]
    return [round(v, 2) for v in dipeptide_percentages]  # Round to two decimal places

# Function to evaluate the model
def evaluate_model(model, X_test, y_test):
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    return accuracy

# Assuming X is a pandas Series containing protein sequences and y is a pandas Series containing labels
# Splitting data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Compute DPC features for each sequence in X_train and X_test
X_train_dpc = np.array(X_train.apply(dpc_compute).tolist())
X_test_dpc = np.array(X_test.apply(dpc_compute).tolist())

print("X_train_dpc shape:", X_train_dpc.shape)
print("y_train shape:", y_train.shape)
print("X_test_dpc shape:", X_test_dpc.shape)
print("y_test shape:", y_test.shape)

# Define hyperparameters for Random Forest
rf_params = {'n_estimators': 100, 'max_depth': 10, 'random_state': 42}

# Train Random Forest model with specified parameters using GPU
rf_model = RandomForestClassifier(n_estimators=rf_params['n_estimators'], max_depth=rf_params['max_depth'], random_state=rf_params['random_state'])
rf_model.fit(X_train_dpc, y_train)

# Evaluate the Random Forest model
rf_accuracy = evaluate_model(rf_model, X_test_dpc, y_test)

# Print complete report for Random Forest
print("\nRandom Forest Model Report using DPC Features only:")
print(f"Parameters: {rf_params}")
print(f"Accuracy: {rf_accuracy}")


X_train_dpc shape: (83868, 400)
y_train shape: (83868,)
X_test_dpc shape: (20968, 400)
y_test shape: (20968,)

Random Forest Model Report using DPC Features only:
Parameters: {'n_estimators': 100, 'max_depth': 10, 'random_state': 42}
Accuracy: 0.9302747043113315


## RF WITH AAC + DPC Combine Feature

In [ ]:
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
from collections import Counter

# Fixed list of 20 standard amino acids
amino_acids = 'ACDEFGHIKLMNPQRSTVWY'

# Function to compute AAC features
def aac_compute(seq):
    seq = seq.replace('^', '')  # Remove '^' character if present
    aa_counts = Counter(seq)
    total_aa = len(seq)
    aa_percentages = [(aa_counts[aa] / total_aa) * 100 if total_aa > 0 else 0 for aa in amino_acids]
    return [round(v, 2) for v in aa_percentages]  # Round to two decimal places

# Function to compute DPC features
def dpc_compute(seq):
    seq = seq.replace('^', '')  # Remove '^' character if present
    dipeptides = [a1 + a2 for a1 in amino_acids for a2 in amino_acids]
    dipeptide_counts = Counter(seq[i:i+2] for i in range(len(seq) - 1))
    total_dipeptides = len(seq) - 1
    dipeptide_percentages = [(dipeptide_counts[dp] / total_dipeptides) * 100 if total_dipeptides > 0 else 0 for dp in dipeptides]
    return [round(v, 2) for v in dipeptide_percentages]  # Round to two decimal places

# Function to evaluate the model
def evaluate_model(model, X_test, y_test):
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    return accuracy

# Assuming X is a pandas Series containing protein sequences and y is a pandas Series containing labels
# Splitting data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Compute AAC and DPC features for each sequence in X_train and X_test
X_train_aac = np.array(X_train.apply(aac_compute).tolist())
X_test_aac = np.array(X_test.apply(aac_compute).tolist())
X_train_dpc = np.array(X_train.apply(dpc_compute).tolist())
X_test_dpc = np.array(X_test.apply(dpc_compute).tolist())

# Concatenate AAC and DPC features
X_train_combined = np.concatenate([X_train_aac, X_train_dpc], axis=1)
X_test_combined = np.concatenate([X_test_aac, X_test_dpc], axis=1)

print("X_train_combined shape:", X_train_combined.shape)
print("y_train shape:", y_train.shape)
print("X_test_combined shape:", X_test_combined.shape)
print("y_test shape:", y_test.shape)

# Define fixed hyperparameters for Random Forest
rf_params = {'n_estimators': 100, 'max_depth': 10, 'random_state': 42}

# Train Random Forest model with fixed parameters using GPU
rf_model = RandomForestClassifier(n_estimators=rf_params['n_estimators'], max_depth=rf_params['max_depth'], random_state=rf_params['random_state'])
rf_model.fit(X_train_combined, y_train)

# Evaluate the Random Forest model
rf_accuracy = evaluate_model(rf_model, X_test_combined, y_test)

# Print complete report for Random Forest
print("\nRandom Forest Model Report:")
print(f"Parameters: {rf_params}")
print(f"Accuracy: {rf_accuracy}")


X_train_combined shape: (83868, 420)
y_train shape: (83868,)
X_test_combined shape: (20968, 420)
y_test shape: (20968,)

Random Forest Model Report:
Parameters: {'n_estimators': 100, 'max_depth': 10, 'random_state': 42}
Accuracy: 0.9302747043113315


## KNN

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
from collections import Counter

# Fixed list of 20 standard amino acids
amino_acids = 'ACDEFGHIKLMNPQRSTVWY'

# Function to compute AAC features
def aac_compute(seq):
    seq = seq.replace('^', '')  # Remove '^' character if present
    aa_counts = Counter(seq)
    total_aa = len(seq)
    aa_percentages = [(aa_counts[aa] / total_aa) * 100 if total_aa > 0 else 0 for aa in amino_acids]
    return [round(v, 2) for v in aa_percentages]  # Round to two decimal places

# Function to compute DPC features
def dpc_compute(seq):
    seq = seq.replace('^', '')  # Remove '^' character if present
    dipeptides = [a1 + a2 for a1 in amino_acids for a2 in amino_acids]
    dipeptide_counts = Counter(seq[i:i+2] for i in range(len(seq) - 1))
    total_dipeptides = len(seq) - 1
    dipeptide_percentages = [(dipeptide_counts[dp] / total_dipeptides) * 100 if total_dipeptides > 0 else 0 for dp in dipeptides]
    return [round(v, 2) for v in dipeptide_percentages]  # Round to two decimal places

# Function to evaluate the model
def evaluate_model(model, X_test, y_test):
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    return accuracy

# Assuming X is a pandas Series containing protein sequences and y is a pandas Series containing labels
# Splitting data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Compute AAC and DPC features for each sequence in X_train and X_test
X_train_aac = np.array(X_train.apply(aac_compute).tolist())
X_test_aac = np.array(X_test.apply(aac_compute).tolist())
X_train_dpc = np.array(X_train.apply(dpc_compute).tolist())
X_test_dpc = np.array(X_test.apply(dpc_compute).tolist())

# Concatenate AAC and DPC features
X_train_combined = np.concatenate([X_train_aac, X_train_dpc], axis=1)
X_test_combined = np.concatenate([X_test_aac, X_test_dpc], axis=1)

print("X_train_combined shape:", X_train_combined.shape)
print("y_train shape:", y_train.shape)
print("X_test_combined shape:", X_test_combined.shape)
print("y_test shape:", y_test.shape)

# Define hyperparameters for KNN
knn_params = {'n_neighbors': 5}

# Train KNN model with fixed parameters
knn_model = KNeighborsClassifier(n_neighbors=knn_params['n_neighbors'])
knn_model.fit(X_train_combined, y_train)

# Evaluate the KNN model
knn_accuracy = evaluate_model(knn_model, X_test_combined, y_test)

# Print complete report for KNN
print("\nK-Nearest Neighbors (KNN) Model Report:")
print(f"Parameters: {knn_params}")
print(f"Accuracy: {knn_accuracy}")


X_train_combined shape: (83868, 420)
y_train shape: (83868,)
X_test_combined shape: (20968, 420)
y_test shape: (20968,)

K-Nearest Neighbors (KNN) Model Report:
Parameters: {'n_neighbors': 5}
Accuracy: 0.9273178176268599


## Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
from collections import Counter

# Fixed list of 20 standard amino acids
amino_acids = 'ACDEFGHIKLMNPQRSTVWY'

# Function to compute AAC features
def aac_compute(seq):
    seq = seq.replace('^', '')  # Remove '^' character if present
    aa_counts = Counter(seq)
    total_aa = len(seq)
    aa_percentages = [(aa_counts[aa] / total_aa) * 100 if total_aa > 0 else 0 for aa in amino_acids]
    return [round(v, 2) for v in aa_percentages]  # Round to two decimal places

# Function to compute DPC features
def dpc_compute(seq):
    seq = seq.replace('^', '')  # Remove '^' character if present
    dipeptides = [a1 + a2 for a1 in amino_acids for a2 in amino_acids]
    dipeptide_counts = Counter(seq[i:i+2] for i in range(len(seq) - 1))
    total_dipeptides = len(seq) - 1
    dipeptide_percentages = [(dipeptide_counts[dp] / total_dipeptides) * 100 if total_dipeptides > 0 else 0 for dp in dipeptides]
    return [round(v, 2) for v in dipeptide_percentages]  # Round to two decimal places

# Function to evaluate the model
def evaluate_model(model, X_test, y_test):
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    return accuracy

# Assuming X is a pandas Series containing protein sequences and y is a pandas Series containing labels
# Splitting data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Compute AAC and DPC features for each sequence in X_train and X_test
X_train_aac = np.array(X_train.apply(aac_compute).tolist())
X_test_aac = np.array(X_test.apply(aac_compute).tolist())
X_train_dpc = np.array(X_train.apply(dpc_compute).tolist())
X_test_dpc = np.array(X_test.apply(dpc_compute).tolist())

# Concatenate AAC and DPC features
X_train_combined = np.concatenate([X_train_aac, X_train_dpc], axis=1)
X_test_combined = np.concatenate([X_test_aac, X_test_dpc], axis=1)

print("X_train_combined shape:", X_train_combined.shape)
print("y_train shape:", y_train.shape)
print("X_test_combined shape:", X_test_combined.shape)
print("y_test shape:", y_test.shape)

# Define hyperparameters for Gradient Boosting
gb_params = {'n_estimators': 100, 'max_depth': 5, 'learning_rate': 0.1, 'random_state': 42}

# Train Gradient Boosting model with specified parameters using GPU
gb_model = GradientBoostingClassifier(n_estimators=gb_params['n_estimators'], max_depth=gb_params['max_depth'], learning_rate=gb_params['learning_rate'], random_state=gb_params['random_state'])
gb_model.fit(X_train_combined, y_train)

# Evaluate the Gradient Boosting model
gb_accuracy = evaluate_model(gb_model, X_test_combined, y_test)

# Print complete report for Gradient Boosting
print("\nGradient Boosting Model Report:")
print(f"Parameters: {gb_params}")
print(f"Accuracy: {gb_accuracy}")


X_train_combined shape: (83868, 420)
y_train shape: (83868,)
X_test_combined shape: (20968, 420)
y_test shape: (20968,)

Gradient Boosting Model Report:
Parameters: {'n_estimators': 100, 'max_depth': 5, 'learning_rate': 0.1, 'random_state': 42}
Accuracy: 0.9297977871041587


## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
from collections import Counter

# Fixed list of 20 standard amino acids
amino_acids = 'ACDEFGHIKLMNPQRSTVWY'

# Function to compute AAC features
def aac_compute(seq):
    seq = seq.replace('^', '')  # Remove '^' character if present
    aa_counts = Counter(seq)
    total_aa = len(seq)
    aa_percentages = [(aa_counts[aa] / total_aa) * 100 if total_aa > 0 else 0 for aa in amino_acids]
    return [round(v, 2) for v in aa_percentages]  # Round to two decimal places

# Function to compute DPC features
def dpc_compute(seq):
    seq = seq.replace('^', '')  # Remove '^' character if present
    dipeptides = [a1 + a2 for a1 in amino_acids for a2 in amino_acids]
    dipeptide_counts = Counter(seq[i:i+2] for i in range(len(seq) - 1))
    total_dipeptides = len(seq) - 1
    dipeptide_percentages = [(dipeptide_counts[dp] / total_dipeptides) * 100 if total_dipeptides > 0 else 0 for dp in dipeptides]
    return [round(v, 2) for v in dipeptide_percentages]  # Round to two decimal places

# Function to evaluate the model
def evaluate_model(model, X_test, y_test):
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    return accuracy

# Assuming X is a pandas Series containing protein sequences and y is a pandas Series containing labels
# Splitting data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Compute AAC and DPC features for each sequence in X_train and X_test
X_train_aac = np.array(X_train.apply(aac_compute).tolist())
X_test_aac = np.array(X_test.apply(aac_compute).tolist())
X_train_dpc = np.array(X_train.apply(dpc_compute).tolist())
X_test_dpc = np.array(X_test.apply(dpc_compute).tolist())

# Concatenate AAC and DPC features
X_train_combined = np.concatenate([X_train_aac, X_train_dpc], axis=1)
X_test_combined = np.concatenate([X_test_aac, X_test_dpc], axis=1)

print("X_train_combined shape:", X_train_combined.shape)
print("y_train shape:", y_train.shape)
print("X_test_combined shape:", X_test_combined.shape)
print("y_test shape:", y_test.shape)

# Define hyperparameters for Logistic Regression
lr_params = {'penalty': 'l2', 'C': 1.0, 'random_state': 42}

# Train Logistic Regression model with specified parameters using GPU
lr_model = LogisticRegression(penalty=lr_params['penalty'], C=lr_params['C'], random_state=lr_params['random_state'])
lr_model.fit(X_train_combined, y_train)

# Evaluate the Logistic Regression model
lr_accuracy = evaluate_model(lr_model, X_test_combined, y_test)

# Print complete report for Logistic Regression
print("\nLogistic Regression Model Report:")
print(f"Parameters: {lr_params}")
print(f"Accuracy: {lr_accuracy}")


X_train_combined shape: (83868, 420)
y_train shape: (83868,)
X_test_combined shape: (20968, 420)
y_test shape: (20968,)

Logistic Regression Model Report:
Parameters: {'penalty': 'l2', 'C': 1.0, 'random_state': 42}
Accuracy: 0.9302747043113315


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
